In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score


In [ ]:
data = pd.read_csv('House_Rent_Dataset.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.describe(include = 'object')

In [ ]:
data.isnull().sum()

In [ ]:
# Separar las columnas de pisos en el piso en el que está y los pisos totales
data["Floor_Number"]=data["Floor"].apply(lambda x:str(x).split()[0])
data["Total_Number_of_Floor"]=data["Floor"].apply(lambda x:str(x).split()[-1])
data.head()

In [ ]:
# Remover columnas no necesarias
data.drop(["Posted On", "Floor", "Area Locality"], axis="columns", inplace=True)
data.head()

In [ ]:
m = data["Floor_Number"]=="Ground"
data[m]

In [ ]:
def fun(c):
    return c.replace("Ground", "0")
clm=["Floor_Number"]
for c in clm:
    data[c]=data[c].apply(fun)
data["Floor_Number"].value_counts()

In [ ]:
def fun2(v):
    return v.replace("Lower", "1")
for v in clm:
    data[v]=data[v].apply(fun2)

In [ ]:
def fun3(b):
    return b
data["Floor_Number"]=data["Floor_Number"].apply(fun3)

In [ ]:
data.drop([2883],axis=0)

In [ ]:
data["Floor_Number"]=pd.to_numeric(data["Floor_Number"], errors='coerce')

In [ ]:
data["Total_Number_of_Floor"].value_counts()

In [ ]:
clm1=["Total_Number_of_Floor"]
for c in clm1:
    data[c]=data[c].apply(fun)
data["Total_Number_of_Floor"]=data["Total_Number_of_Floor"].astype(int)

In [ ]:
data['Floor_Number'].fillna(0, inplace = True)

In [ ]:
data["Floor_Number"]=data["Floor_Number"].astype(int)

In [ ]:
data = pd.get_dummies(data, columns=['Area Type', 'City', 'Furnishing Status', 'Tenant Preferred', 'Point of Contact'])
data.head()

In [ ]:
data.corr()['Rent'].sort_values(ascending=False)

In [ ]:
X = data.drop(columns=['Rent'], axis=1)
y = data['Rent']

In [ ]:
X, y = X.astype('int64'),y.astype('int64')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

In [ ]:
y_train = np.array(y_train).reshape(y_train.shape[0], 1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()
X_train = mms_X.fit_transform(X_train)
y_train = mms_y.fit_transform(y_train)

In [ ]:
model = RandomForestRegressor(n_estimators=700, max_depth=70, min_samples_leaf=3, min_samples_split= 5,random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(mms_X.transform(X_test.values))

In [ ]:
y_pred = mms_y.inverse_transform(y_pred.reshape(len(y_test), 1))
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.values.reshape(len(y_test),1)),1))

In [ ]:
r2_score(y_pred, y_test)